# Classify propositions from a call for project

We will get a webpage, extract projects propositions from it and then do a topic extraction to get a sens of group of proposition.

# Scrapping

In [58]:
url = 'https://gnius.esante.gouv.fr/-liste-des-structures-fournisseurs-de-solutions-numeriques-en-recherche-d-un-binome-pour-candidater-a-l-appel-a-projets-'

from bs4 import BeautifulSoup
import requests
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

In [59]:
import pandas as pd
import re
df = pd.DataFrame()
for article in soup.find_all(class_='content_article'):
    titre_article = article.find(class_='titre_article').get_text()
    texte_article = article.find(class_='texte_article').get_text()
    note = article.find(class_='note_coeur note').get_text()
    start = texte_article.find('tre mis en contact avec ') + len('contact avec ')
    if texte_article.find('structure', start):
        type_entite = 'structure'
    elif texte_article.find('acteur', start):
        type_entite = 'acteur'
    else:
        type_entite = ''
    texte_article = texte_article[:texte_article.find('Vous souhaitez être')]
    one_article = {'Entite':titre_article, 'Projet': texte_article, 'Type' : type_entite, 'Note': note}
    df = df.append(one_article, ignore_index=True)
    

In [60]:
df.head()

,Entite,Note,Projet,Type
0,CENTRE HOSPITALIER DU MARIN,2,"Situé sur le territoire sud de la Martinique, ...",structure
1,EHPAD LES HAUTS DE PLAISANCE,4,Nous avons déjà développé la télémédecine et l...,structure
2,CHI LYS HYROME,5,Nous avons plusieurs besoins :\n nous souhaito...,structure
3,EHPAD LES HIRONDELLES,3,EHPAD DE 55 LITS - crée en 1995. Inscrire l’eh...,structure
4,EHPAD LA PIERRE,3,Région : Pays-de-la-Loire\nVille : St Pierre d...,structure


In [61]:
df.to_csv('appel-a-projets.csv', sep=';')

# Analyse

In [64]:
data = df.Projet.array

In [69]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import nltk
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=nltk.corpus.stopwords.words('french'), lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation( max_iter=10, learning_method='online') # n_topics=NUM_TOPICS,
lda_Z = lda_model.fit_transform(data_vectorized)
 


[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1] 1.0


In [67]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [70]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  32.161891
0       0.648955  71.707100       2        1  15.604403
5      71.019684   9.971006       3        1   9.701147
3       3.785981  33.185135       4        1   9.466502
4     -42.666500  -9.222070       5        1   6.970137
7      -3.621689 -43.526516       6        1   6.821256
9      -3.632402  -4.171708       7        1   6.485443
8     -36.584400  35.804413       8        1   5.545934
1      44.391380  50.615383       9        1   4.419072
6      40.235809 -31.591665      10        1   2.824215, topic_info=              Term       Freq      Total Category  logprob  loglift
193           sant  62.000000  62.000000  Default  30.0000  30.0000
206          soins  30.000000  30.000000  Default  29.0000  29.0000
81           ehpad  22.000000  22.000000  Default  28.0000  28.0000
141        patient  25.000000  25.000000  Default  27.0000  27.0000
129            num  29.000000  29.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
108            int   0.699806  19.222275  Topic10  -4.7423   0.2539
8             afin   0.683343  13.943752  Topic10  -4.7661   0.5511
102  ile-de-france   0.683940  17.047837  Topic10  -4.7652   0.3510
227            tre   0.688355  27.911028  Topic10  -4.7588  -0.1355
212     structures   0.677253  21.801492  Topic10  -4.7750   0.0953

[456 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.219535        acc
0         2  0.274419        acc
0         3  0.054884        acc
0         4  0.054884        acc
0         6  0.164651        acc
...     ...       ...        ...
238       8  0.130563        vie
238       9  0.065281        vie
239       1  0.089294  virtuelle
239       2  0.535761  virtuelle
239       7  0.178587  virtuelle

[1122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 4, 5, 8, 10, 9, 2, 7])